#### **NEM Review contract co-design workshop**

# Contract financial performance modelling

## Prepare data

### Access generators from latest Generation Information

In [1]:
# Python libraries
import matplotlib.pyplot as plt, matplotlib as mpl, numpy as np
import os, pandas as pd, seaborn as sns, statsmodels.api as sm
from sqlalchemy import create_engine, text
from Scripts.functions import python_setup, get_mms_data

# Set up Python
working_dir, charts_dir, data_dir = python_setup()

# Get generator information
gen_info = pd.read_excel(
    os.path.join(data_dir, "NEM Generation Information July 2025.xlsx"),
    sheet_name=4,
    header=1,
    usecols=[0, 1, 2, 5, 6],
    names=["Region", "Status", "Name", "Technology", "DUID"])

# Filter and reformat
gen_info = gen_info[    
    # Existing stations only
    (gen_info["Status"] == "Existing Plant") & 
    # SA only
    (gen_info["Region"] == "SA1") & 
    # Wind and solar only
    (gen_info["Technology"].isin(["Wind - Wind", "Solar - Solar"]))
].iloc[:, [2, 3, 4]]
# Simplify technology name
gen_info["Technology"] = gen_info["Technology"].map(lambda t: t.split(" - ")[1])

# Remove duplicate DUIDs
gen_info.drop_duplicates(subset="DUID", keep="first", inplace=True)

# Reorder by DUID
gen_info.set_index("DUID", inplace=True)

# Display DUIDs used in analysis
for tech in gen_info["Technology"].value_counts().index:
    print(f"{tech}: {gen_info["Technology"].value_counts().loc[tech]} generators")

Wind: 26 generators
Solar: 19 generators


### Collect historic DUID data for South Australia

Execute SQL query to collect output, availability and maximum capacity data for each DUID.

In [2]:
# SA DUID data path
sa_duid_data_path = os.path.join(data_dir, "sa_duid_data.csv")

# Check if file exists
if not os.path.exists(sa_duid_data_path):
    print("sa_duid_data.csv not found. Querying database ...")

    # Define DUIDs of interest
    duid_list = gen_info.index

    # Get generation and price data
    sa_duid_data = get_mms_data(
        script_name="collect_sa_duid_data",
        arguments={
            "start_date": "2020-01-01",
            "end_date": "2024-12-31",
            "duid_list": ",".join([f"'{d}'" for d in duid_list])})

    # Save data to file
    sa_duid_data.to_csv(sa_duid_data_path, index=False)    
    print(f"Data saved successfully.")

else:
    print(f"Loading existing sa_duid_data.csv file ...")
    # Load existing data
    sa_duid_data = pd.read_csv(
        sa_duid_data_path,
        parse_dates=[0])

# Add name and technology
print("Joining with gen info data ...")
sa_duid_data["Technology"] = sa_duid_data["DUID"].map(lambda d: gen_info.at[d, "Technology"])
sa_duid_data["Name"] = sa_duid_data["DUID"].map(lambda d: gen_info.at[d, "Name"])

# Display market data
print("Data loaded successfully.")
sa_duid_data

Loading existing sa_duid_data.csv file ...
Joining with gen info data ...
Data loaded successfully.


,Interval,DUID,Output,Availability,Maximum capacity,Relative Output,Relative availability,Technology,Name
0,2020-01-01 00:00:00,BLUFF1,0.0590,0.118,53.0,0.001113,0.002226,Wind,Hallett 5 The Bluff WF
1,2020-01-01 00:00:00,BNGSF1,0.0000,0.000,110.0,0.000000,0.000000,Solar,Bungala One Solar Farm
2,2020-01-01 00:00:00,BNGSF2,0.0000,0.000,110.0,0.000000,0.000000,Solar,Bungala Two Solar Farm
3,2020-01-01 00:00:00,CLEMGPWF,33.7193,33.294,57.0,0.591567,0.584105,Wind,Clements Gap Wind Farm
4,2020-01-01 00:00:00,HALLWF1,1.6255,1.351,95.0,0.017111,0.014221,Wind,Hallett Stage 1 Brown Hill
...,...,...,...,...,...,...,...,...,...
16953403,2024-12-31 23:55:00,TB2SF1,0.0000,0.000,87.0,0.000000,0.000000,Solar,Tailem Bend Stage 2 Solar Project
16953404,2024-12-31 23:55:00,TBSF1,0.0000,0.000,95.0,0.000000,0.000000,Solar,Tailem Bend - Solar
16953405,2024-12-31 23:55:00,WATERLWF,0.2500,0.000,130.0,0.001923,0.000000,Wind,Waterloo Wind Farm
16953406,2024-12-31 23:55:00,WGWF1,0.6000,0.700,119.0,0.005042,0.005882,Wind,Willogoleche Wind Farm


### Collect whole-of-market data for South Australia

Execute SQL query to collect SA prices and total demand data.

In [3]:
# SA demand and prices path
sa_demand_and_prices_path = os.path.join(data_dir, "sa_demand_and_prices.csv")

# Check if file exists
if not os.path.exists(sa_demand_and_prices_path):
    print("sa_demand_and_prices.csv not found. Querying database ...")

    # Get generation and price data
    sa_demand_and_prices = get_mms_data(
        script_name="collect_sa_demand_and_prices",
        arguments={
            "start_date": "2020-01-01",
            "end_date": "2024-12-31"})
    sa_demand_and_prices.set_index("Interval", inplace=True)

    # Save data to file
    sa_demand_and_prices.to_csv(sa_demand_and_prices_path)
    print(f"Data saved successfully.")

else:
    print(f"Loading existing sa_demand_and_prices.csv file ...")
    # Load existing data
    sa_demand_and_prices = pd.read_csv(
        sa_demand_and_prices_path,
        parse_dates=[0],
        index_col=0)

# Display market data
print("Data loaded successfully.")
sa_demand_and_prices

Loading existing sa_demand_and_prices.csv file ...
Data loaded successfully.


,Total demand,Price
Interval,,
2020-01-01 00:00:00,1466.53,68.00000
2020-01-01 00:05:00,1422.33,68.00000
2020-01-01 00:10:00,1425.70,68.50000
2020-01-01 00:15:00,1423.95,68.50000
2020-01-01 00:20:00,1389.36,68.50000
...,...,...
2024-12-31 23:35:00,1375.14,130.16134
2024-12-31 23:40:00,1369.37,141.79538
2024-12-31 23:45:00,1372.97,130.58007


## Observations of SA wind and solar market

Calculate and visualise metrics of:
* Size of the wind and solar fleets over time (2020 to 2024)
* Capacity factors of wind and solar generators
* Intraday shape of wind and solar generation, and load
* Demand-weighted prices for SA (whole of market), wind and solar

### Size of wind and solar fleets

In [11]:
# Identify changes in DUID capacity:
for duid in gen_info.index:
    name = gen_info.at[duid, "Name"]
    duid_data = sa_duid_data[sa_duid_data["DUID"] == duid]
    capacity_readings = len(duid_data["Maximum capacity"].unique())
    if capacity_readings > 1:
        print(f"{duid} ({name}) has {capacity_readings} values of maximum capacity:")


### Demand-weighted prices

In [4]:
sa_demand_and_prices["Total value"] = sa_demand_and_prices.iloc[:, :2].product(axis=1)
sa_demand_and_prices_qtr = sa_demand_and_prices.resample("QE").sum()
sa_demand_and_prices_qtr["DWA price"] = sa_demand_and_prices_qtr["Total value"] / sa_demand_and_prices_qtr["Total demand"]
sa_demand_and_prices_qtr

,Total demand,Price,Total value,DWA price
Interval,,,,
2020-03-31,31861965.50,1.698033e+06,2.689440e+09,84.409108
2020-06-30,34965022.76,1.056503e+06,1.592212e+09,45.537288
2020-09-30,36281156.39,1.068019e+06,1.735669e+09,47.839405
2020-12-31,29057682.24,7.615153e+05,1.047735e+09,36.057057
2021-03-31,30338791.58,1.051573e+06,1.678646e+09,55.330015
2021-06-30,34028230.67,1.831248e+06,2.721546e+09,79.979064
2021-09-30,34836572.12,1.364122e+06,2.315254e+09,66.460429
2021-12-31,27214737.04,1.083267e+06,1.744908e+09,64.116295
2022-03-31,33264893.85,1.829420e+06,2.905269e+09,87.337389
